# Setup

In [1]:
from piper import piper
from piper.verbs import *
from piper.defaults import *

piper version 0.0.7, last run: Friday, 26 February 2021 15:06:09


# Example: Melt/pivot data (pivot_longer)

In [2]:
csv_file = 'inputs/FAO.csv'
df = pd.read_csv(csv_file, encoding = 'latin1')
head(df, 3)

21477 rows, 63 columns


,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,1928.0,1904.0,1666.0,1950.0,2001.0,1808.0,2053.0,2045.0,2154.0,1819.0,1963.0,2215.0,2310.0,2335.0,2434.0,2512.0,2282.0,2454.0,2443.0,2129.0,2133.0,2068.0,1994.0,1851.0,1791.0,1683.0,2194.0,1801.0,1754.0,1640.0,1539.0,1582.0,1840.0,1855.0,1853.0,2177.0,2343.0,2407.0,2463.0,2600.0,2668.0,2776.0,3095.0,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,183.0,183.0,182.0,220.0,220.0,195.0,231.0,235.0,238.0,213.0,205.0,233.0,246.0,246.0,255.0,263.0,235.0,254.0,270.0,259.0,248.0,217.0,217.0,197.0,186.0,200.0,193.0,202.0,191.0,199.0,197.0,249.0,218.0,260.0,319.0,254.0,326.0,347.0,270.0,372.0,411.0,448.0,460.0,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,76.0,76.0,76.0,76.0,76.0,75.0,71.0,72.0,73.0,74.0,71.0,70.0,72.0,76.0,77.0,80.0,60.0,65.0,64.0,64.0,60.0,55.0,53.0,51.0,48.0,46.0,46.0,47.0,46.0,43.0,43.0,40.0,50.0,46.0,41.0,44.0,50.0,48.0,43.0,26.0,29.0,70.0,48.0,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360


## melt/pivot_longer

In [3]:
id_vars = df.loc[:,'Area Abbreviation':'longitude'].columns.tolist()
df2 = df.melt(id_vars=id_vars)
head(df2)

1138281 rows, 12 columns


,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,variable,value
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,Y1961,1928.0
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,Y1961,183.0
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,Y1961,76.0
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,Y1961,237.0


In [4]:
%%piper

df2
>> where("Area.isin(['Italy', 'Ireland']) & variable.isin(['Y2000', 'Y2013'])")
>> group_by(['Area', 'variable'])
>> summarise(Total=pd.NamedAgg('value', 'sum')).unstack()
>> adorn().astype(int)

Total        
          Y2000   Y2013
Ireland   13383   18160
Italy    164746  153450
All      178129  171610

In [5]:
%%piper

df2
>> where("Area.isin(['Italy', 'Ireland']) & variable.isin(['Y2000', 'Y2013'])")
>> group_by(['Item', 'variable'])
>> summarise(Total=pd.NamedAgg('value', 'sum')).unstack()
>> adorn().astype(int)
>> flatten_cols(remove_prefix='Total')
>> assign(ratio=lambda x: round(x.Y2013*100/x.Y2000, 2))

,Y2000,Y2013,ratio
Alcoholic Beverages,5453,4432,81.28
Animal fats,781,778,99.62
Apples and products,1335,1063,79.63
"Aquatic Animals, Others",0,27,inf
Aquatic Plants,0,0,NaN
...,...,...,...
"Vegetables, Other",7297,6808,93.30
Wheat and products,11093,11707,105.54
Wine,3099,1939,62.57
Yams,0,1,inf
